<a href="https://colab.research.google.com/github/BF667/Nooshell-RVC/blob/main/Nooshell_RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nooshell RVC

This notebook sets up and runs the Nooshell RVC application in Google Colab with GPU acceleration. Follow the steps below to install dependencies, configure tunneling, and launch the UI.

**Author**: [BF667](https://github.com/BF667)  
**Project Repository**: [Nooshell-RVC](https://github.com/Politrees-Kanoyo/Nooshell-RVC)

## Installation

Run this cell to install dependencies and clone the repository. Optionally mount Google Drive to save or load data.

In [ ]:
#@title Install Dependencies
from IPython.display import clear_output, display
from google.colab import drive
import os
import subprocess

def check_gpu():
    try:
        !nvidia-smi
        return True
    except:
        print("Warning: No GPU detected. This notebook requires a GPU for optimal performance.")
        return False

def install_dependencies():
    print("Cloning repository...")
    repo_url = "https://github.com/Politrees-Kanoyo/Nooshell-RVC.git"
    try:
        subprocess.run(["git", "clone", repo_url], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except subprocess.CalledProcessError:
        print("Error: Failed to clone repository. Please check the URL or your network connection.")
        return False

    print("Installing Python dependencies...")
    requirements_path = "/content/Nooshell-RVC/requirements.txt"
    try:
        subprocess.run(["pip", "install", "-r", requirements_path], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except subprocess.CalledProcessError:
        print("Error: Failed to install Python dependencies.")
        return False

    print("Updating apt and installing libcudnn8...")
    try:
        subprocess.run(["apt-get", "update"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        subprocess.run(["apt-get", "install", "-y", "libcudnn8"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except subprocess.CalledProcessError:
        print("Error: Failed to install system dependencies.")
        return False

    return True

#@markdown Check to mount Google Drive (optional)
use_drive = False  #@param {type:"boolean"}

print("Checking GPU availability...")
check_gpu()

if use_drive:
    print("Mounting Google Drive...")
    try:
        drive.mount('/content/drive', force_remount=True)
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")

print("Starting installation...")
if install_dependencies():
    clear_output()
    print("Installation completed successfully!")
else:
    print("Installation failed. Please check the error messages above.")

## Run the Application

This cell sets up a tunneling service to access the Nooshell RVC UI and launches the application. Select a tunneling service and provide necessary credentials if required.

**Instructions**:
- Choose a tunnel type from the dropdown.
- For Ngrok, LocalTunnel, or Horizon, wait for the Local URL to appear before accessing the Public URL.
- Ensure you have the required tokens/IDs for Ngrok or Horizon if selected.

In [ ]:
#@title Launch UI
import codecs
import os
import subprocess
import time
import urllib.request

path = "/content/Nooshell-RVC"
os.chdir(path)
app = codecs.decode('ncc.cl', 'rot_13')

#@markdown Select the tunneling service
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudflare", "LocalTunnel", "Horizon"]

#@markdown For Ngrok: Get your authtoken from https://dashboard.ngrok.com/tunnels/authtokens/new
ngrok_authtoken = "" #@param {type:"string"}

#@markdown Select Ngrok region for lower latency
ngrok_region = "us - United States (Ohio)" #@param ["au - Australia (Sydney)", "eu - Europe (Frankfurt)", "ap - Asia/Pacific (Singapore)", "us - United States (Ohio)", "jp - Japan (Tokyo)", "in - India (Mumbai)", "sa - South America (Sao Paulo)"]

#@markdown For Horizon: Get your ID from https://hrzn.run/dashboard/
horizon_id = "" #@param {type:"string"}

def setup_tunnel(tunnel_type, ngrok_token, ngrok_reg, horizon_id):
    if tunnel_type == "Gradio":
        return "--share", None
    
    if tunnel_type == "Ngrok":
        if not ngrok_token:
            raise ValueError("Ngrok authtoken is required.")
        from pyngrok import conf, ngrok
        ngrok_config = conf.PyngrokConfig(auth_token=ngrok_token, region=ngrok_reg[:2])
        conf.set_default(ngrok_config)
        http_tunnel = ngrok.connect(4000, bind_tls=True)
        print(f"Ngrok Tunnel Public URL: {http_tunnel.public_url}")
        return "", http_tunnel.public_url
    
    if tunnel_type == "Cloudflare":
        print("Setting up Cloudflare tunnel...")
        !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
        !dpkg -i cloudflared-linux-amd64.deb
        !rm -f nohup.out
        !nohup cloudflared tunnel --url localhost:4000 &
        time.sleep(5)
        cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
        if cloudflare_url:
            print(f"Cloudflare Tunnel Public URL: {cloudflare_url[0]}")
            return "", cloudflare_url[0]
        else:
            raise RuntimeError("Failed to retrieve Cloudflare URL.")
    
    if tunnel_type == "LocalTunnel":
        print("Setting up LocalTunnel...")
        !npm install -g localtunnel
        with open('url.txt', 'w') as file:
            file.write('')
        get_ipython().system_raw('lt --port 4000 >> url.txt 2>&1 &')
        time.sleep(4)
        endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
        with open('url.txt', 'r') as file:
            tunnel_url = file.read().replace("your url is: ", "")
        print(f"LocalTunnel Public URL: {tunnel_url}")
        print(f"LocalTunnel Password: {endpoint_ip}")
        return "", tunnel_url
    
    if tunnel_type == "Horizon":
        if not horizon_id:
            raise ValueError("Horizon ID is required.")
        print("Setting up Horizon tunnel...")
        !npm install -g @hrzn/cli
        !hrzn login $horizon_id
        with open('url.txt', 'w') as file:
            file.write('')
        get_ipython().system_raw('hrzn tunnel http://localhost:4000 >> url.txt 2>&1 &')
        time.sleep(4)
        tunnel_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url.txt
        if tunnel_url:
            print(f"Horizon Tunnel Public URL: {tunnel_url[0]}")
            return "", tunnel_url[0]
        else:
            raise RuntimeError("Failed to retrieve Horizon URL.")

try:
    # Clean up any existing processes
    !fuser -k 4000/tcp
    !fuser -k 9999/tcp

    # Set up tunnel
    share_flag, public_url = setup_tunnel(Tunnel, ngrok_authtoken, ngrok_region, horizon_id)

    # Launch the application
    print("Launching Nooshell RVC...")
    command = f"python {app} {share_flag}"
    subprocess.run(command, shell=True, check=True)
except Exception as e:
    print(f"Error: {e}")
    print("Failed to launch the application. Please check the error message and ensure all inputs are correct.")

## Contributors

Thanks to everyone who contributed to the Nooshell RVC project:  
<a href="https://github.com/Politrees-Kanoyo/Nooshell-RVC/graphs/contributors" target="_blank">
  <img src="https://contrib.rocks/image?repo=Politrees-Kanoyo/Nooshell-RVC" />
</a>